In [1]:
print("coucou")

coucou

In [2]:
using Polyhedra
using JuMP
using LinearAlgebra
using Gurobi #, Cbc, GLPK
using Plots
using JLD
using Combinatorics

In [3]:
include("../lib/linear_system_v3.jl")
print("Done")

Done

In [4]:
include("../lib/double_integrator_drones.jl")
print("Done")

Done

In [5]:
include("../lib/utils_MILP.jl");
print("Done")

Done

In [6]:
# data of the problem
T=20
N_m=3
N_c=4

enforceRegular=false
minimizeScaling=false #enforceRegular

tested_numDrones=[1,1,4]

3-element Array{Int64,1}:
 1
 1
 4

In [14]:
len=length(tested_numDrones)

instantiation_times=zeros(len)
optimization_times=zeros(len)
feasibilities=zeros(len)

for i=1:len
    # data of the problem
    num_drones=tested_numDrones[i]
    
    println("\n====== num drones=$num_drones ======\n")
    
    # Instantiation (including loading the problem)
    time_beginning=time_ns()
    
    system,_=get_drones(num_drones,T);
    model,C_bar,S_bar, P_z_all,z_tilde, P_u_all, u_tilde, polEta=instantiateASAP(system,N_m,N_c,T; minimizeScaling=minimizeScaling,enforceRegular=enforceRegular);
    
    time_ending=time_ns()
    instantiation_time=(time_ending-time_beginning)/1e9
    print("\n==== Instantiation time = $(instantiation_time) seconds ====\n")
    instantiation_times[i]=instantiation_time
    
    # Set an optimozer and solve the problem
    set_optimizer(model, Gurobi.Optimizer)
    
    # Solving the problem
    time_beginning=time_ns()
    
    optimize!(model)
    
    time_ending=time_ns()
    optimization_time=(time_ending-time_beginning)/1e9
    print("\n\n==== Optimization time = $(optimization_time) seconds ====\n\n")
    optimization_times[i]=optimization_time
    
    feasibilities[i] = (termination_status(model) == MOI.TerminationStatusCode(1)) # 1 if feasible
end

numberSchedulings=length(combinations(0:T-1,N_m))*length(combinations(0:T-1,N_c))


====== num drones=1 ======


==== Instantiation time = 6.007019461 seconds ====
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 68414 rows, 42352 columns and 844676 nonzeros
Model fingerprint: 0x2da61585
Variable types: 42312 continuous, 40 integer (40 binary)
Coefficient statistics:
  Matrix range     [1e-03, 1e+06]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 2e+01]
Presolve removed 55740 rows and 23592 columns
Presolve time: 1.28s
Presolved: 12674 rows, 18760 columns, 137680 nonzeros
Variable types: 18720 continuous, 40 integer (40 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   27975    0.0000000e+00   7.473069e+00   3.190854e+08      5s
Concurrent spin time: 0.61s

Solved with dual simplex

Root relaxation: objective 0

5523300

In [15]:
# save results
io = open("results/dimensionScalingAnalysis.txt", "w") do io
    println(io,"numberSchedulings=$numberSchedulings")
    println(io,"num_drones, instantiation_time [sec], optimization_times [sec], feasibility")
    for i=1:len
        print(io, tested_numDrones[i])
        print(io,", ")
        print(io, instantiation_times[i])
        print(io,", ")
        print(io, optimization_times[i])
        print(io,", ")
        print(io, feasibilities[i])
        println(io)
    end
end

In [16]:
instantiation_times

3-element Array{Float64,1}:
  6.007019461
  6.337903608
 63.219990374

In [17]:
optimization_times

3-element Array{Float64,1}:
 315.546002019
 312.094911157
 332.744406006

In [18]:
numberSchedulings

5523300

In [19]:
feasibilities

3-element Array{Float64,1}:
 1.0
 1.0
 1.0